<a href="https://colab.research.google.com/github/ugur-sa/Bachelorarbeit/blob/main/SentimentTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hole HTML-Dokument der Nachricht über dessen Link und bereinige es grob

In [4]:
from bs4 import BeautifulSoup
import requests

res = requests.get("https://finance.yahoo.com/news/us-stocks-p-500-wobbles-210042198.html")

soup = BeautifulSoup(res.text, "html.parser")

title = soup.find("title").text
print(title)

soup = soup.find("div", attrs={'class':'caas-body'})

US STOCKS-S&P 500 wobbles to lower close on mixed Fed messages, strong GDP data


Hier wird weiter gearbeitet mit dem Div-Element, dass den Nachrichtentext enthält. Hierbei wird der pure Text extrahiert und Listen, Buttons und die Tags selber entfernt.

In [ ]:
for div in soup.find_all('div'):
    div.unwrap()

for ul in soup.find_all('ul'):
    # Finde das unmittelbar vorhergehende <p>-Tag, wenn vorhanden
    p_tag = ul.find_previous_sibling('p')

    # Wenn ein <p>-Tag gefunden wurde, entferne es
    if p_tag:
        p_tag.decompose()

# Entferne alle <div>-Elemente
for ul in soup.find_all("ul"):
    ul.decompose()

for button in soup.find_all("button"):
    button.decompose()

for p_tag in soup.find_all('p', string="©2023 Bloomberg L.P."):
    p_tag.decompose()

# text_lines = [p.get_text() + '\n' for p in soup.find_all('p')]

for p in soup.find_all('p'):
  print(p.text)

# print(text_lines)

Klassifiziere die Nachricht mit einem vortrainierten Modell mit der Huggingsface Bibliothek und PyTorch

In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

model_name = "nickmuchi/distilroberta-finetuned-financial-text-classification"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Dein Text
text = ""

for p in soup.find_all('p'):
  text = text + p.text + "\n"
# Tokenisiere den Text
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Wende das Modell an
with torch.no_grad():
    outputs = model(**inputs)

# Hole die Modellvorhersagen
predictions = outputs.logits

# Konvertiere Vorhersagen in Wahrscheinlichkeiten und hole das maximale
predictions = torch.nn.functional.softmax(predictions, dim=1)
predicted_class = torch.argmax(predictions).item()

# Zuordnung der Klassen
classes = ["bearish", "neutral", "bullish"]

# Ergebnis
print(f"Sentiment: {classes[predicted_class]}")


tokenizer_config.json:   0%|          | 0.00/355 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/894 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

Sentiment: bullish
